## Luria Delbruck Spontaneous Mutations

In [5]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

np.random.seed(123)

def deterministic_growth_population(initial_population, initial_time, max_time, growth_rate):
    """
    The population of sensitive bacterial cells at the end time.
    
    Parameters:
    max_time: End time of the simulation.
    initial_population: Initial population size.
    growth_rate: Growth rate per individual.
    
    Returns:
    the total populations at the end time.
    """
    
    # sensitive cells grow exponentially
    return initial_population * np.exp(growth_rate*max_time)

def yule_process(initial_population, initial_time, max_time, growth_rate):
    """
    The growth of mutants populations 
    from initial time until end time by Yule process.
    
    Parameters:
    max_time: End time of the simulation.
    initial_population: Initial population size.
    growth_rate: Growth rate per individual.
    
    Returns:
    times: a list of time.
    population:  a list of populations.
    """
    
    time = initial_time
    population = initial_population
    times = [initial_time]
    populations = [initial_population]
    
    while time < max_time:
        
        # generate random time with rate n*cell division rate
        next_time = np.random.exponential(1 / (population * growth_rate))
        
        # population increase by 1
        time += next_time
        population += 1
        
        times.append(time)
        populations.append(population)
        
    return times, populations

def cumulative_intensity(t, initial_population, mutation_rate, growth_rate):
    """
    Parameters:
    t: Time.
    initial_population: Initial population size.
    growth_rate: Growth rate per individual.
    
    Returns:
    Cumulative intensity (integral of intensity function from 0 to t).
    """
    
    # Cumulative intensity is the integral of mutation_rate * initial_population * exp(growth_rate * u) du from 0 to t
    return (mutation_rate * initial_population / growth_rate) * (np.exp(growth_rate * t) - 1)

def generate_mutation_events(end_time, initial_population, growth_rate, mutation_rate):
    """
    Generate a list of time that mutation occurs.
    
    Parameters:
    end_time: The time at which to stop generating mutation events.
    initial_population: The initial population size.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    A list of mutation event times.
    """
    mutation_events = []
    current_time = 0
    
    while current_time < end_time:
        
        U = np.random.uniform(0, 1)
            
        # Solve for the next event time using the inverse transform method
        # Find time t such that cumulative_intensity(t) = cumulative_intensity(current_time) + log(1/U)
        current_intensity = cumulative_intensity(current_time, initial_population, mutation_rate, growth_rate)
        target_intensity = current_intensity + np.log(1/U)
        
        # Solve for the next time using the inverse of the cumulative intensity
        next_time = (1 / growth_rate) * np.log((growth_rate * target_intensity / (mutation_rate * initial_population)) + 1)
        
        if next_time <= end_time:
            mutation_events.append(next_time)
        
        current_time = next_time
    
    return mutation_events

def yule_process_with_mutations(end_time, initial_population, growth_rate, mutation_rate):
    """
    Mutants that occurs at set of mutation event growth with Yule process
    
    Parameters:
    end_time: The time at which to evaluate the population.
    initial_population: The initial population size.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    final_population: Total number of sensitve bacterial populations.
    total_mutant_population: Total number of mutant populations.
    mutation_events: A list of mutation event times.
    """
    # Calculate final population at the end time for original population
    population_at_t = deterministic_growth_population(initial_population, 0, end_time, growth_rate)
    final_population = population_at_t
    
    # Generate mutation events using the continuous-time inverse transform method
    mutation_events = generate_mutation_events(end_time, initial_population, growth_rate, mutation_rate)
    
    # Simulate the growth of mutants initiated at mutation events
    total_mutant_population = 0
    for mutation_time in mutation_events:
        
        # Calculate the mutant population at the end time using Yule process
        time_mutant, pop_mutant = yule_process(initial_population, mutation_time, end_time, growth_rate)
        mutant_final_population = pop_mutant[-1]
        total_mutant_population += mutant_final_population
    
    return final_population, total_mutant_population, mutation_events

def run_simulations(num_simulations, end_time, initial_population, growth_rate, mutation_rate):
    """
    Parameters:
    num_simulations: The number of simulations to run.
    end_time: The time at which to evaluate the population.
    initial_population: The initial population size.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    expected_mutant_population: Expected number of total mutant populations.
    variance_mutant_population: Variance number of total mutant populations.
    p0: Probability that no mutations have occured over time.
    expected_population: Expected number of sensitive bacterial populations.
    mutant_populations: List of number of total mutant populations for all simulations.
    final_populations: List of number of sensitive bacterial populations for all simulations.
    """
    final_populations = []
    mutant_populations = []
    mut_event = []
    no_mutation = 0
    
    for _ in tqdm(range(num_simulations), desc="Running simulations"):
        final_population, total_mutant_population, mutation_events = yule_process_with_mutations(
            end_time, initial_population, growth_rate, mutation_rate)
        
        final_populations.append(final_population)
        mutant_populations.append(total_mutant_population)
        mut_event.append(len(mutation_events))
        
        # a simulation with no mutants
        if total_mutant_population == 0:
            no_mutation += 1
    
    expected_mutant_population = np.mean(mutant_populations)
    variance_mutant_population = np.var(mutant_populations)
    p0 = no_mutation/num_simulations
    expected_population = np.mean(final_populations)
    
    return expected_mutant_population, variance_mutant_population, p0, expected_population, mutant_populations, final_populations

In [7]:
if __name__ == "__main__":
    end_time = 10
    initial_population = 1
    growth_rate = 1
    mutation_rate = 0.0001
    num_simulations = 1000

    expected_mutant_population, variance_mutant_population, p0, expected_population, mutant_populations, final_populations = run_simulations(
        num_simulations, end_time, initial_population, growth_rate, mutation_rate)

Running simulations: 100%|███████████████| 1000/1000 [00:00<00:00, 21344.19it/s]


#### Expected value and variance of the number of mutant populations

In [8]:
def theory_expected_mutant(mutation_rate, growth_rate, end_time):
    """
    The expected number of mutant population from Luria and Delbruck's Legacy.
    
    Parameters:
    end_time: The time at which to stop generating mutation events.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    Expected number of mutant population from Luria and Delbruck's Legacy.
    """
    return mutation_rate * end_time * np.exp(growth_rate * end_time)

def theory_var_mutant(mutation_rate, growth_rate, end_time):
    """
    The variance of mutant population from Luria and Delbruck's Legacy.
    
    Parameters:
    end_time: The time at which to stop generating mutation events.
    growth_rate: The growth rate of the population.
    mutation_rate: Mutation rate per bacterium per unit time.
    
    Returns:
    Variance of mutant population from Luria and Delbruck's Legacy.
    """
    return (((mutation_rate * np.exp(growth_rate * end_time)) / growth_rate) * (2 * np.exp(growth_rate * end_time) - growth_rate * end_time - 2))